# **sys.dm\_db\_index\_operational\_stats**

## O problema da fragmentação de Indices

<span style="color: rgb(22, 22, 22); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">A fragmentação de índice ocorre quando a ordenação lógica nas páginas de índice não corresponde à ordenação física. As páginas podem estar desordenadas durante instruções de modificação de dados de rotina, como&nbsp;</span>  `UPDATE`<span style="color: rgb(22, 22, 22); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">,&nbsp;</span>  `DELETE`  <span style="color: rgb(22, 22, 22); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">&nbsp;e&nbsp;</span>  `INSERT`<span style="color: rgb(22, 22, 22); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">. A fragmentação pode apresentar problemas de desempenho devido à E/S adicional necessária para localizar os dados que estão sendo referenciados pelos ponteiros dentro das páginas de índice.</span>

<span style="color:rgb(22, 22, 22);font-family:&quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif;font-size:16px;background-color:rgb(255, 255, 255);"><b>Uma recompilação</b> pode estar online ou offline, dependendo do comando executado ou da edição do SQL Server que está sendo usada. Um processo de recompilação offline removerá e recriará o índice. Se você puder fazer isso online, um novo índice será criado em paralelo ao índice existente. Depois de criar o índice, o índice existente será descartado e o novo será renomeado para corresponder ao nome do antigo</span>

## **Estrutura da DMV `sys.dm_db_index_operational_stats`**

In [ ]:
sys.dm_db_index_operational_stats (
    { database_id | NULL | 0 | DEFAULT },
    { object_id | NULL | 0 | DEFAULT },
    { index_id | NULL | 0 | -1 | DEFAULT },
    { partition_number | NULL | 0 | -1 | DEFAULT }
)


### Parâmetros

### 

- **database\_id**: ID do banco de dados. Use `NULL`, `0`, ou `DEFAULT` para todos os bancos de dados.
- **object\_id**: ID da tabela ou visualização indexada. Use `NULL`, `0`, ou `DEFAULT` para todos os objetos.
- **index\_id**: ID do índice. Use `NULL`, `0`, `-1`, ou `DEFAULT` para todos os índices.
- **partition\_number**: Número da partição. Use `NULL`, `0`, `-1`, ou `DEFAULT` para todas as partições

## Colunas retornadas

###   

### 

A DMV retorna várias colunas que fornecem informações detalhadas sobre as operações nos índices:

- **database\_id**: ID do banco de dados.
- **object\_id**: ID da tabela ou visualização indexada.
- **index\_id**: ID do índice.
- **partition\_number**: Número da partição.
- **leaf\_insert\_count**: Número de inserções de registros na folha do índice.
- **leaf\_delete\_count**: Número de exclusões de registros na folha do índice.
- **leaf\_update\_count**: Número de atualizações de registros na folha do índice.
- **leaf\_ghost\_count**: Número de registros "fantasmas" na folha do índice (registros marcados para exclusão).
- **nonleaf\_insert\_count**: Número de inserções de registros nos níveis não folha do índice.
- **nonleaf\_delete\_count**: Número de exclusões de registros nos níveis não folha do índice.
- **nonleaf\_update\_count**: Número de atualizações de registros nos níveis não folha do índice.
- **rows\_read**: Número de linhas lidas.
- **row\_lock\_count**: Número de bloqueios de linhas.
- **row\_lock\_wait\_in\_ms**: Tempo total de espera para bloqueios de linhas, em milissegundos.
- **page\_lock\_count**: Número de bloqueios de páginas.
- **page\_lock\_wait\_in\_ms**: Tempo total de espera para bloqueios de páginas, em milissegundos.
- **index\_lock\_promotion\_attempts**: Número de tentativas de promoção de bloqueio de índice.
- **index\_lock\_promotion\_count**: Número de promoções de bloqueio de índice bem-sucedidas.

### Exemplos de Uso

### 

1. **Obter Estatísticas Operacionais para Todos os Índices em uma Tabela Específica**

In [ ]:
SELECT *
FROM sys.dm_db_index_operational_stats(DB_ID(), OBJECT_ID('MinhaTabela'), NULL, NULL);


> **2\. Obter Estatísticas Operacionais para um Índice Específico**:

In [ ]:
SELECT *
FROM sys.dm_db_index_operational_stats(DB_ID(), OBJECT_ID('MinhaTabela'), 1, NULL);


> **3.<span style="color: var(--vscode-foreground);">Obter Estatísticas Operacionais para Todos os Bancos de Dados e Índices</span>**

In [ ]:
SELECT *
FROM sys.dm_db_index_operational_stats(NULL, NULL, NULL, NULL);


### Análise e Uso Prático

- **Diagnóstico de Performance**: Usar `sys.dm_db_index_operational_stats` para entender a frequência de inserções, atualizações e deleções pode ajudar a identificar índices que estão sob alta carga e podem se beneficiar de ajustes ou manutenção.
- **Identificação de Índices Ineficientes**: Estatísticas como `rows_read` e `row_lock_wait_in_ms` ajudam a identificar índices que podem estar causando contenção ou atrasos significativos.
- **Ajustes de Fill Factor**: Informações sobre inserções, deleções e atualizações podem guiar a configuração de fatores de preenchimento (fill factors) apropriados para reduzir a fragmentação e melhorar a performance.
- **Manutenção Proativa**: Monitorar os valores de `leaf_ghost_count` pode ajudar a determinar quando uma reconstrução ou reorganização de índices é necessária para remover registros "fantasmas" e melhorar a eficiência do índice.